## 1. |mp0rt

In [1]:
# importing all the libraries needed for project, should all be in Pipfile

from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
import datetime
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [2]:
# import listings.csv, set the index to the listing id, and turn off truncation for df display
df_list = pd.read_csv('listings.csv').set_index('id')
pd.set_option('display.max_columns', 500)

## 2. Wr4ng|

In [3]:
def wrangle(X):
    """A function to clean our data"""
    
    #Make a copy of the df
    X = X.copy()

    ## DROP unwanted columns; HiCard, urls, empty, redundant, etc.           80>|
    drop_cols=['listing_url', 'scrape_id', 'last_scraped', 'name', 'description', 
               'neighborhood_overview', 'picture_url', 'host_url',
               'host_name', 'host_since', 'host_location', 'host_about', 
               'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 
               'neighbourhood', 'neighbourhood_group_cleansed', 'bathrooms', 
               'amenities', 'calendar_updated', 'has_availability',
               'calendar_last_scraped', 'minimum_minimum_nights', 
               'maximum_minimum_nights', 'minimum_maximum_nights', 
               'maximum_maximum_nights']
    X.drop(columns=drop_cols, inplace=True)
    
    ## A FUNC and execution thereof tallying number of host contacts that have been verified.
    def num_contact(items):
        items = items.replace('[','').replace(']','').replace("'",'').split(',')
        return len(items)
    X['host_verifications'] = X['host_verifications'].apply(num_contact)
    
    ## Changing columns to BOOL vals     
    # Convert "t/f" columns to BOOl columns.
    def strToBool(text):
      if text == 't':
        return True
      else:
        return False
    X['host_has_profile_pic'] = X['host_has_profile_pic'].apply(strToBool)
    X['host_identity_verified'] = X['host_identity_verified'].apply(strToBool)
    X['host_is_superhost'] = X['host_is_superhost'].apply(strToBool)
    X['instant_bookable'] = X['instant_bookable'].apply(strToBool)
    
    # Convert license column to BOOL column.
    def licenseToBool(text):      
        if type(text) != float: 
            return True
        else:
            return False
    X['license'] = X['license'].apply(lambda x: licenseToBool(x))  
    
    
    ## Shorten/rename neighbourhood_cleansed' column as 'neighborhood'
    X = X.rename({'neighbourhood_cleansed' : 'neighborhood'}, axis=1)
    
    ### BATHROOM FUNCTIONS
    ## Change 'bathroom_text' column name to 'num_bathroom', i.e. number of baths
    X = X.rename({'bathrooms_text' : 'num_bathroom'}, axis=1)
    # Drop 2 null values in the 'num_bathroom' column
    X = X[X['num_bathroom'].isnull() == False]
    # Engineer a feature to specify if baths are shared or not
    def parse_baths_shared(string):
        if 'shared' in string.lower(): # .lower because there is one value to remove
            return True                # "Shared Half Bath", we do
        else:
            return False
    X['shared_bath'] = X['num_bathroom'].apply(parse_baths_shared)
    
    # Parse floats from the bathrooms_text column
    def parse_baths(text):
        if 'half-bath' in text.lower():
            return 0.5

        text = text.split(' ')
        return float(text[0])
    X['num_baths'] = X['num_bathroom'].apply(parse_baths)
    X = X.drop(columns='num_bathroom')
    ### END BATH
    
    # Change 'price' data type from OBJECT to INT
    X['price'] = X['price'].apply(lambda x: float(x.replace('$',"").replace(',','')))
    
    # Change the data types of columns 'first_review' and 'last_review'
    def datestojd(dt_str):
        if type(dt_str) == float:
            return dt_str
        else:
            jul_date = pd.to_datetime(dt_str)
        return jul_date.to_julian_date()
    X['first_review'] = X['first_review'].apply(lambda x: datestojd(x))
    X['last_review'] = X['last_review'].apply(lambda x: datestojd(x))
        
    return X

## €dα


In [ ]:
# Create a df for pandas profiling
df_profile = wrangle(df_list)

In [ ]:
# Using pandas profiling to look at features and their properties
profile = ProfileReport(df_profile)
profile.to_notebook_iframe()

# m0d3|

In [4]:
# Create a df for train/test split and randomforestregssor models
df_model = wrangle(df_list)

# Train//tesT :: :: :: SΦLi_|_

In [5]:
# setting the target for train/val/test splits
target = 'price'

y_init = df_model[target]
X_init = df_model.drop(columns=target)

In [6]:
print(X_init.shape)
print(y_init.shape)

(3151, 47)
(3151,)


In [7]:
X_subseq, X_test, y_subseq, y_test = train_test_split(X_init, y_init, test_size=0.1, random_state=13)

In [8]:
print(X_test.shape)

(316, 47)


In [9]:
print(X_subseq.shape)
print(y_subseq.shape)

(2835, 47)
(2835,)


In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_subseq, y_subseq, test_size=0.1114, random_state=17)

In [11]:
X_val.shape

(316, 47)

In [17]:
print(X_train.shape)
print(X_val.shape)

(2519, 47)
(316, 47)


## B4SSLI3N

In [18]:
baseline_pred = [y_train.mean()] * len(y_train)

baseline_mae = mean_absolute_error(y_train,baseline_pred)

print(baseline_mae)

82.89001996891902


## PIGH:P:LINE

### GrdSRCH M0de|

In [13]:
pipe_01 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(random_state = 13,
                           n_jobs = -2)
)

# params are a DICT
params_01 = {
    # 'key__hyperparam'(which step in pipeline you want looked at) : 'value'     
    'randomforestregressor__max_depth' : range(200, 1201, 100),
    'randomforestregressor__n_estimators' :  range(50, 351, 10),
    'randomforestregressor__max_samples' : np.arange(0.1, 0.9, 0.1)
    }

In [ ]:
model_01 = GridSearchCV(
    pipe_01,
    param_grid = params_01,
    n_jobs = -2,
    cv = 5,
    verbose = 3
)

In [14]:
pipe_01.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['host_response_time',
                                      'host_response_rate',
                                      'host_acceptance_rate', 'neighborhood',
                                      'property_type', 'room_type'],
                                mapping=[{'col': 'host_response_time',
                                          'data_type': dtype('O'),
                                          'mapping': within an hour        1
within a few hours    2
a few days or more    3
within a day          4
NaN                   5
dtype: int64},
                                         {'col': 'host_response_rate',
                                          'data_type': dtyp...
Private room in cottage              35
Shared room in apartment             36
Room in hotel                        37
Private room in camper/rv            38
Private room in hostel               39
Private room                         4

In [16]:
importances = pipe_01.named_steps["randomforestregressor"].feature_importances_

features = X_train.columns
feature_importances = pd.Series(importances, index=features).sort_values()

feature_importances.tail(20)

review_scores_rating      0.015461
review_scores_checkin     0.015807
property_type             0.016095
availability_365          0.017377
availability_30           0.017386
reviews_per_month         0.017642
first_review              0.019246
minimum_nights            0.020169
availability_90           0.022007
beds                      0.023287
availability_60           0.025618
accommodates              0.025731
longitude                 0.027004
host_id                   0.028576
latitude                  0.029103
minimum_nights_avg_ntm    0.047569
bedrooms                  0.054194
neighborhood              0.058383
number_of_reviews         0.085911
num_baths                 0.243179
dtype: float64

## Stripped Model

Currently top 10 importance features

In [ ]:
# secondary model with top 10 feats

X_train2 = X_train[['host_id', 'last_review', 'minimum_nights', 'latitude',
                   'minimum_nights_avg_ntm', 'first_review', 'bedrooms',
                   'number_of_reviews', 'neighborhood', 'num_baths']]

X_val2 = X_val[['host_id', 'last_review', 'minimum_nights', 'latitude',
                'minimum_nights_avg_ntm', 'first_review', 'bedrooms',
                'number_of_reviews', 'neighborhood', 'num_baths']]

In [ ]:
pipe_02 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(
        n_estimators = 250,
        max_depth = 150,
        max_samples = 0.5,
        random_state = 13,
        n_jobs = -2)
)

In [ ]:
import joblib

filename = 'working_model.sav'
joblib.dump(pipe_02, filename)

### Grid search for tuning

In [ ]:
X_train3 = X_train[['review_scores_rating', 'review_scores_checkin', 'property_type',
       'availability_365', 'availability_30', 'reviews_per_month',
       'first_review', 'minimum_nights', 'availability_90', 'beds',
       'availability_60', 'accommodates', 'longitude', 'host_id', 'latitude',
       'minimum_nights_avg_ntm', 'bedrooms', 'neighborhood',
       'number_of_reviews', 'num_baths']]

X_val3 = X_val[['review_scores_rating', 'review_scores_checkin', 'property_type',
       'availability_365', 'availability_30', 'reviews_per_month',
       'first_review', 'minimum_nights', 'availability_90', 'beds',
       'availability_60', 'accommodates', 'longitude', 'host_id', 'latitude',
       'minimum_nights_avg_ntm', 'bedrooms', 'neighborhood',
       'number_of_reviews', 'num_baths']]

In [ ]:
pipe_03 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(random_state = 13,
                           n_jobs = -2)
)

# params are a DICT
params_03 = {
    # 'key__hyperparam'(which step in pipeline you want looked at) : 'value'     
    'randomforestregressor__max_depth' : range(50, 200, 10),
    'randomforestregressor__n_estimators' :  range(350, 550, 10),
    'randomforestregressor__max_samples' : np.arange(0.05, 0.5, 0.05)
    }

In [ ]:
model_03 = GridSearchCV(
    pipe_03,
    param_grid = params_03,
    n_jobs = -2,
    cv = 5,
    verbose = 3
)

In [ ]:
model_03.fit(X_train3, y_train)

In [ ]:
model_03.best_params_

In [ ]:
print('Training MAE:', mean_absolute_error(y_train, model_03.predict(X_train3)))
print('Validation MAE:', mean_absolute_error(y_val, model_03.predict(X_val3)))

In [ ]:
params_03_2 = {
    # 'key__hyperparam'(which step in pipeline you want looked at) : 'value'     
    'randomforestregressor__max_depth' : range(20, 150, 10),
    'randomforestregressor__n_estimators' :  range(350, 550, 10),
    'randomforestregressor__max_samples' : np.arange(0.05, 0.5, 0.05)
    }

In [ ]:
# doubling folds, slight tweak down to max_depth
model_03_2 = GridSearchCV(
    pipe_03,
    param_grid = params_03_2,
    n_jobs = -2,
    cv = 10,
    verbose = 3
)

In [ ]:
model_03_2.fit(X_train3, y_train)

In [ ]:
model_03_2.best_params_

In [ ]:
print('Training MAE:', mean_absolute_error(y_train, model_03_2.predict(X_train3)))
print('Validation MAE:', mean_absolute_error(y_val, model_03_2.predict(X_val3)))

## Even finer tuning

In [19]:
pipe_04 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(random_state = 13,
                           n_jobs = -2)
)

# params are a DICT
params_04 = {
    # 'key__hyperparam'(which step in pipeline you want looked at) : 'value'     
    'randomforestregressor__max_depth' : range(10, 70, 5),
    'randomforestregressor__n_estimators' :  range(480, 600, 5),
    'randomforestregressor__max_samples' : np.arange(0.05, 0.13, 0.025)
    }

In [ ]:
model_04 = GridSearchCV(
    pipe_04,
    param_grid = params_04,
    n_jobs = -2,
    cv = 5,
    verbose = 3
)

In [ ]:
model_04.fit(X_train3, y_train)

In [ ]:
print('Training MAE:', mean_absolute_error(y_train, model_04.predict(X_train3)))
print('Validation MAE:', mean_absolute_error(y_val, model_04.predict(X_val3)))

In [ ]:
model_04.best_params_

In [20]:
params_04_2 = {
    # 'key__hyperparam'(which step in pipeline you want looked at) : 'value'     
    'randomforestregressor__max_depth' : range(20, 40, 4),
    'randomforestregressor__n_estimators' :  range(480, 530, 5),
    'randomforestregressor__max_samples' : np.arange(0.025, 0.1, 0.025)
    }

In [21]:
model_04_2 = GridSearchCV(
    pipe_04,
    param_grid = params_04_2,
    n_jobs = -2,
    cv = 10,
    verbose = 3
)

In [ ]:
model_04_2.fit(X_train3, y_train)

In [ ]:
print('Training MAE:', mean_absolute_error(y_train, model_04_2.predict(X_train3)))
print('Validation MAE:', mean_absolute_error(y_val, model_04_2.predict(X_val3)))

In [ ]:
model_04_2.best_params_

In [ ]:
best_v_01 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(
        n_estimators = 250,
        max_depth = 150,
        max_samples = 0.5,
        random_state = 13,
        n_jobs = -2)
)

In [ ]:
best_v_01.fit(X_train3, y_train)

In [ ]:
import joblib

filename = 'best_model_v_01.sav'
joblib.dump(best_v_01, filename)